# Using the qiskit interface to perform mGST

In this notebook we will generate random sequences of gates, create qiskit circuits from them, and simulate them on a qiskit simulator.
Then we estimate the gates.

In [11]:
%load_ext autoreload
%autoreload 2

import numpy as np

from qibo import gates, set_backend
from qibo.backends import NumpyBackend

from mGST import qibo_interface
from mGST.qibo_interface import job_counts_to_mgst_format, get_gate_estimation

backend = NumpyBackend()
set_backend("numpy")

[Qibo 0.2.12|INFO|2024-09-16 10:20:37]: Using numpy backend on /CPU:0


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Example usage with RX, RY, and RZZ gates
rx_pi_2 = gates.RX(0, np.pi / 2)
rx_pi = gates.RX(0, np.pi)
ry_pi_2 = gates.RY(0, np.pi / 2)
ry_pi = gates.RY(0, np.pi)
rzz_pi_2 = gates.RZZ(0, 1, np.pi / 2)
idle_gate = gates.I(0)

gate_set = [idle_gate, rx_pi, ry_pi, rx_pi_2, ry_pi_2,]
K_true = qibo_interface.qibo_gate_to_operator(gate_set)

In [13]:
sequence_number = 100  # Number of sequences
shots = 1000  # Number of samples per sequences
sequence_length = 7 # The depth of randomly generated circuits for GST
n_qubits = 1 # The total number of qubits on the backend
active_qubits = [0] # The qubits on which the gate set is defined
n_povm = 2**(len(active_qubits)) # The number of measurement outcomes

In [14]:
# Running the circuits and formating the result
gate_sequences = qibo_interface.get_gate_sequence(sequence_number, sequence_length, len(gate_set))
qibo_circuits = qibo_interface.get_qibo_circuits(gate_sequences, gate_set, n_qubits, active_qubits)
results = backend.execute_circuits(qibo_circuits, nshots=shots)
counts = [result.frequencies() for result in results]
sequence_results = job_counts_to_mgst_format(active_qubits, n_povm, counts)

In [15]:
# Running GST
X_opt, E_opt, rho_opt = get_gate_estimation(gate_set, gate_sequences, sequence_results, shots, rK = 4)

Starting mGST optimization...
  0%|          | 0/100 [00:01<?, ?it/s]
Batch optimization successful, improving estimate over full data....
 12%|█▏        | 6/50 [00:09<01:10,  1.61s/it]
	 Convergence criterion satisfied
	 Final objective 1.35e-4 with # of initializations: 1 in time 10.87s
Mean variation error: 0.04166070624365831
Optimizing gauge...
Compressive GST routine complete
First results:
           F_avg  Min. Spectral distances
Gate 0  0.999520                 0.005017
Gate 1  0.999242                 0.004768
Gate 2  0.997904                 0.003151
Gate 3  0.999947                 0.007859
Gate 4  0.999109                 0.002212
                                
Final cost function     0.000135
Mean TVD estimate-data  0.009410
Mean TVD target-data    0.010080
SPAM error:             0.001743
